# Creating and modelling metallic supercells

In this section we will be concerned with modelling supercells of aluminium.
When dealing with periodic problems there is no unique definition of the
lattice: Clearly any duplication of the lattice along an axis is also a valid
repetitive unit to describe exactly the same system.
This is exactly what a **supercell** is: An $n$-fold repetition along one of the
axes of the original lattice.

The following code achieves this for aluminium:

In [1]:
using DFTK
using LinearAlgebra
using ASEconvert

function aluminium_setup(repeat=1; Ecut=7.0, kgrid=[2, 2, 2])
    a = 7.65339
    lattice = a * Matrix(I, 3, 3)
    Al = ElementPsp(:Al; psp=load_psp("hgh/lda/al-q3"))
    atoms     = [Al, Al, Al, Al]
    positions = [[0.0, 0.0, 0.0], [0.0, 0.5, 0.5], [0.5, 0.0, 0.5], [0.5, 0.5, 0.0]]
    unit_cell = periodic_system(lattice, atoms, positions)

    # Make supercell in ASE:
    # We convert our lattice to the conventions used in ASE, make the supercell
    # and then convert back ...
    supercell_ase = convert_ase(unit_cell) * pytuple((repeat, 1, 1))
    supercell     = pyconvert(AbstractSystem, supercell_ase)

    # Unfortunately right now the conversion to ASE drops the pseudopotential information,
    # so we need to reattach it:
    supercell = attach_psp(supercell; Al="hgh/lda/al-q3")

    # Construct an LDA model and discretise
    # Note: We disable symmetries explicitly here. Otherwise the problem sizes
    #       we are able to run on the CI are too simple to observe the numerical
    #       instabilities we want to trigger here.
    model = model_LDA(supercell; temperature=1e-3, symmetries=false)
    PlaneWaveBasis(model; Ecut, kgrid)
end;

    CondaPkg Found dependencies: /home/runner/.julia/packages/ASEconvert/CNQ1A/CondaPkg.toml
    CondaPkg Found dependencies: /home/runner/.julia/packages/PythonCall/qTEA1/CondaPkg.toml
    CondaPkg Dependencies already up to date


As part of the code we are using a routine inside the ASE,
the [atomistic simulation environment](https://wiki.fysik.dtu.dk/ase/index.html)
for creating the supercell and make use of the two-way interoperability of
DFTK and ASE. For more details on this aspect see the documentation
on Input and output formats.

Write an example supercell structure to a file to plot it:

In [2]:
setup = aluminium_setup(5)
convert_ase(periodic_system(setup.model)).write("al_supercell.png")

┌ Warning: Skipping atomic property pseudopotential, which is not supported in ASE.
└ @ ASEconvert ~/.julia/packages/ASEconvert/CNQ1A/src/ASEconvert.jl:123
┌ Warning: Skipping atomic property pseudopotential, which is not supported in ASE.
└ @ ASEconvert ~/.julia/packages/ASEconvert/CNQ1A/src/ASEconvert.jl:123


Python: None

<img src="https://docs.dftk.org/stable/examples/al_supercell.png" width=500 height=500 />

As we will see in this notebook the modelling of a system generally becomes
harder if the system becomes larger.

- This sounds like a trivial statement as *per se* the cost per SCF step increases
  as the system (and thus $N$) gets larger.
- But there is more to it:
  If one is not careful also the *number of SCF iterations* increases
  as the system gets larger.
- The aim of a proper computational treatment of such supercells is therefore
  to ensure that the **number of SCF iterations remains constant** when the
  system size increases.

For achieving the latter DFTK by default employs the `LdosMixing`
preconditioner [^HL2021] during the SCF iterations. This mixing approach is
completely parameter free, but still automatically adapts to the treated
system in order to efficiently prevent charge sloshing. As a result,
modelling aluminium slabs indeed takes roughly the same number of SCF iterations
irrespective of the supercell size:

[^HL2021]:
   M. F. Herbst and A. Levitt.
   *Black-box inhomogeneous preconditioning for self-consistent field iterations in density functional theory.*
   J. Phys. Cond. Matt *33* 085503 (2021). [ArXiv:2009.01665](https://arxiv.org/abs/2009.01665)

In [3]:
self_consistent_field(aluminium_setup(1); tol=1e-4);

┌ Warning: Skipping atomic property pseudopotential, which is not supported in ASE.
└ @ ASEconvert ~/.julia/packages/ASEconvert/CNQ1A/src/ASEconvert.jl:123
n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -8.298236076532                   -0.85    5.0         
  2   -8.300182383562       -2.71       -1.26    1.6    123ms
  3   -8.300432088642       -3.60       -1.89    3.0    144ms
  4   -8.300461507305       -4.53       -2.75    1.6    120ms
  5   -8.300464152322       -5.58       -3.08    3.1    172ms
  6   -8.300464393674       -6.62       -3.27    8.5    282ms
  7   -8.300464515029       -6.92       -3.42    1.8    130ms
  8   -8.300464575768       -7.22       -3.54    2.8    145ms
  9   -8.300464625436       -7.30       -3.75    3.1    170ms
 10   -8.300464636201       -7.97       -3.89    2.6    165ms
 11   -8.300464643430       -8.14       -4.21    2.9    214ms


In [4]:
self_consistent_field(aluminium_setup(2); tol=1e-4);

┌ Warning: Skipping atomic property pseudopotential, which is not supported in ASE.
└ @ ASEconvert ~/.julia/packages/ASEconvert/CNQ1A/src/ASEconvert.jl:123
n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -16.67432358081                   -0.70    5.9         
  2   -16.67858264367       -2.37       -1.14    2.0    342ms
  3   -16.67920232200       -3.21       -1.86    3.0    342ms
  4   -16.67927518135       -4.14       -2.73    3.5    354ms
  5   -16.67928551510       -4.99       -3.17    5.8    467ms
  6   -16.67928617613       -6.18       -3.45    2.4    338ms
  7   -16.67928621561       -7.40       -3.90    2.0    301ms
  8   -16.67928622074       -8.29       -4.53    2.2    321ms


In [5]:
self_consistent_field(aluminium_setup(4); tol=1e-4);

┌ Warning: Skipping atomic property pseudopotential, which is not supported in ASE.
└ @ ASEconvert ~/.julia/packages/ASEconvert/CNQ1A/src/ASEconvert.jl:123
n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -33.32715215479                   -0.56    7.4         
  2   -33.33439104941       -2.14       -1.00    1.5    1.08s
  3   -33.33596938278       -2.80       -1.72    6.8    1.50s
  4   -33.33609779872       -3.89       -2.62    4.1    1.32s
  5   -33.33675577823       -3.18       -2.20    8.2    1.81s
  6   -33.33679095335       -4.45       -2.25    1.4    996ms
  7   -33.33694252674       -3.82       -3.28    1.6    1.16s
  8   -33.33694364389       -5.95       -3.60    4.0    1.54s
  9   -33.33694369381       -7.30       -3.67    1.5    990ms
 10   -33.33694375960       -7.18       -4.14    1.4    954ms


When switching off explicitly the `LdosMixing`, by selecting `mixing=SimpleMixing()`,
the performance of number of required SCF steps starts to increase as we increase
the size of the modelled problem:

In [6]:
self_consistent_field(aluminium_setup(1); tol=1e-4, mixing=SimpleMixing());

┌ Warning: Skipping atomic property pseudopotential, which is not supported in ASE.
└ @ ASEconvert ~/.julia/packages/ASEconvert/CNQ1A/src/ASEconvert.jl:123
n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -8.298552575124                   -0.85    5.0         
  2   -8.300272866518       -2.76       -1.59    1.2   93.5ms
  3   -8.300438397022       -3.78       -2.73    2.8    116ms
  4   -8.300455422047       -4.77       -2.76    5.1    184ms
  5   -8.300464465901       -5.04       -3.34    1.0   93.1ms
  6   -8.300464618161       -6.82       -3.90    1.8    139ms
  7   -8.300464642461       -7.61       -4.48    2.8    128ms


In [7]:
self_consistent_field(aluminium_setup(4); tol=1e-4, mixing=SimpleMixing());

┌ Warning: Skipping atomic property pseudopotential, which is not supported in ASE.
└ @ ASEconvert ~/.julia/packages/ASEconvert/CNQ1A/src/ASEconvert.jl:123
n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -33.31516466511                   -0.56    6.6         
  2   -33.28385068287   +   -1.50       -1.24    2.4    937ms
  3   +9.041647761096   +    1.63       -0.25    6.8    2.10s
  4   -33.32512633076        1.63       -1.67    6.4    1.96s
  5   -33.18312340284   +   -0.85       -1.24    3.8    1.56s
  6   -33.21999492799       -1.43       -1.52    3.5    1.40s
  7   -32.98361776728   +   -0.63       -1.29    4.4    1.34s
  8   -33.33522135481       -0.45       -2.24    4.5    1.35s
  9   -33.33681621604       -2.80       -2.65    3.1    1.24s
 10   -33.33678251141   +   -4.47       -2.66    2.9    1.25s
 11   -33.33691257341       -3.89       -2.95    1.8    1.02s
 12   -33.33693266982       -4.70      

For completion let us note that the more traditional `mixing=KerkerMixing()`
approach would also help in this particular setting to obtain a constant
number of SCF iterations for an increasing system size (try it!). In contrast
to `LdosMixing`, however, `KerkerMixing` is only suitable to model bulk metallic
system (like the case we are considering here). When modelling metallic surfaces
or mixtures of metals and insulators, `KerkerMixing` fails, while `LdosMixing`
still works well. See the Modelling a gallium arsenide surface example
or [^HL2021] for details. Due to the general applicability of `LdosMixing` this
method is the default mixing approach in DFTK.